In [ ]:
"""
Make generate images that distort the existing crab images.
These images are saved to a file and will later be used to train the model.
Distort by: color, haze, gausian noise, motion blur, brightness, roation, scale

Code by Anika and Claude, 2025, for MATE
"""

In [1]:
import cv2
import numpy as np
import os
from pathlib import Path

In [ ]:
class UnderwaterAugmentation:
    """
    Generate realistic underwater image augmentations for green crab detection.
    Based on underwater distortion characteristics from scientific literature.
    """
    
    def __init__(self, output_dir='augmented_images'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        
    def add_underwater_color_distortion(self, img, depth_factor=0.5):
        """
        Simulate wavelength-dependent light absorption.
        Red light is absorbed quickly, blue/green penetrates deeper.
        
        Args:
            depth_factor: 0-1, higher values simulate deeper water
        """
        img_float = img.astype(np.float32)
        
        # Reduce red channel (absorbed first)
        img_float[:,:,2] *= (1.0 - depth_factor * 0.6)
        
        # Slightly reduce green
        img_float[:,:,1] *= (1.0 - depth_factor * 0.3)
        
        # Boost blue slightly (penetrates deepest)
        img_float[:,:,0] *= (1.0 + depth_factor * 0.2)
        
        # Add blue-green tint
        blue_green_tint = np.full_like(img_float, [20, 10, 0]) * depth_factor
        img_float = cv2.add(img_float, blue_green_tint)
        
        return np.clip(img_float, 0, 255).astype(np.uint8)
    
    def add_underwater_haze(self, img, haze_strength=0.3):
        """
        Simulate light scattering and haze from suspended particles.
        """
        # Create atmospheric light (bluish-white)
        atmospheric_light = np.array([180, 170, 150])
        
        # Blend image with atmospheric light
        haze_layer = np.full_like(img, atmospheric_light, dtype=np.float32)
        img_float = img.astype(np.float32)
        
        hazy = cv2.addWeighted(img_float, 1-haze_strength, 
                               haze_layer, haze_strength, 0)
        
        return hazy.astype(np.uint8)
    
    def add_gaussian_noise(self, img, mean=0, sigma=15):
        """Add Gaussian noise to simulate sensor noise in low light."""
        noise = np.random.normal(mean, sigma, img.shape).astype(np.float32)
        noisy = img.astype(np.float32) + noise
        return np.clip(noisy, 0, 255).astype(np.uint8)
    
    def add_motion_blur(self, img, kernel_size=5):
        """Simulate blur from water movement or camera motion."""
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
        kernel = kernel / kernel_size
        
        # Random blur direction
        angle = np.random.randint(0, 180)
        M = cv2.getRotationMatrix2D((kernel_size/2, kernel_size/2), angle, 1)
        kernel = cv2.warpAffine(kernel, M, (kernel_size, kernel_size))
        
        return cv2.filter2D(img, -1, kernel)
    
    def adjust_brightness(self, img, factor=1.0):
        """Adjust brightness to simulate different depths/lighting."""
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
        hsv[:,:,2] *= factor
        hsv[:,:,2] = np.clip(hsv[:,:,2], 0, 255)
        return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    
    def generate_augmented_dataset(self, image_path, num_variations=20):
        """
        Generate comprehensive augmented dataset.
        
        Args:
            image_path: path to original crab image
            num_variations: Number of augmented versions
        """
        img = cv2.imread(str(image_path))
        if img is None:
            print(f"Warning: Could not load {image_path}")
            return
        
        base_name = Path(image_path).stem
        print(f"Processing {base_name}...")
        
        # Save original
        cv2.imwrite(str(self.output_dir / f"{base_name}_original.jpg"), img)
        
        # Generate variations
        for i in range(num_variations):
            augmented = img.copy()
            aug_name = f"{base_name}_aug_{i:03d}"
            
            # Random rotation (-180 to 180 degrees)
            if np.random.random() > 0.3:
                angle = np.random.randint(-180, 180)
                augmented = self.rotate_image(augmented, angle)
                aug_name += f"_rot{angle}"
            
            # Random scale (0.5 to 1.5)
            if np.random.random() > 0.3:
                scale = np.random.uniform(0.5, 1.5)
                augmented = self.scale_image(augmented, scale)
                aug_name += f"_scl{scale:.2f}"
            
            # Crop to content after rotation/scaling
            augmented = self.crop_to_content(augmented, margin=20)
            
            # Underwater color distortion (varying depths)
            if np.random.random() > 0.2:
                depth = np.random.uniform(0.2, 0.8)
                augmented = self.add_underwater_color_distortion(augmented, depth)
                aug_name += f"_depth{depth:.2f}"
            
            # Haze/turbidity
            if np.random.random() > 0.3:
                haze = np.random.uniform(0.1, 0.5)
                augmented = self.add_underwater_haze(augmented, haze)
                aug_name += f"_haze{haze:.2f}"
            
            # Brightness variation
            if np.random.random() > 0.3:
                brightness = np.random.uniform(0.6, 1.4)
                augmented = self.adjust_brightness(augmented, brightness)
                aug_name += f"_bri{brightness:.2f}"
            
            # Gaussian noise
            if np.random.random() > 0.4:
                sigma = np.random.randint(5, 20)
                augmented = self.add_gaussian_noise(augmented, sigma=sigma)
                aug_name += f"_gnoise{sigma}"
            
            # Motion blur
            if np.random.random() > 0.5:
                kernel_size = np.random.choice([3, 5, 7])
                augmented = self.add_motion_blur(augmented, kernel_size)
                aug_name += f"_blur{kernel_size}"
            
            # Save augmented image
            output_path = self.output_dir / f"{aug_name}.jpg"
            cv2.imwrite(str(output_path), augmented)
        
        print(f"  Generated {num_variations} variations for {base_name}")
        
        print(f"\nAugmentation complete! Images saved to: {self.output_dir}")
        total_images = len(list(self.output_dir.glob("*.jpg")))
        print(f"Total images: {total_images}")

In [16]:
# Initialize augmentation pipeline
augmenter = UnderwaterAugmentation(output_dir='augmented_crabs')

# List of your original crab images
image_files = ['greenCrab.jpg', 'jonahCrab.jpeg', 'rockCrab.jpg']

# Check if files exist
existing_files = [f for f in image_files if os.path.exists("crabImages/"+f)]

if not existing_files:
    print("Error: No crab images found!")
    print("Please ensure 'greenCrab.jpg', 'jonahCrab.jpg', 'rockCrab.jpg' are in the current directory.")

In [13]:
augmenter.generate_augmented_dataset('crabImages/greenCrab.jpg', num_variations=30)

Processing greenCrab...
  Generated 30 variations for greenCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 0


In [14]:
augmenter.generate_augmented_dataset('crabImages/jonahCrab.jpeg', num_variations=30)

Processing jonahCrab...
  Generated 30 variations for jonahCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 0


In [15]:
augmenter.generate_augmented_dataset('crabImages/rockCrab.jpg', num_variations=30)

Processing rockCrab...
  Generated 30 variations for rockCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 0
